In [1]:
import tangram as tg
import squidpy as sq
import scanpy as sc

##  在细胞水平使用Tangram

In [ ]:
# ad_sp = sc.read_h5ad('/home/pxy/home/pxy/Tangram/data/test_ad_sp.h5ad')
# ad_sc = sc.read_h5ad('/home/pxy/home/pxy/Tangram/data/test_ad_sc.h5ad')


In [2]:
# 加载空间 Visium 数据
adata_sp = sq.datasets.visium_fluo_adata_crop()

# 加载对应的小鼠皮层 scRNA-seq 数据
adata_sc = sq.datasets.sc_mouse_cortex()


HTTPError: HTTP Error 403: Forbidden

pp_adatas 寻找sc数据和st数据的基因交集，保存在两个adatas的uns中用于后续的映射和分析<br>
genes参数用来自定义交集基因的子集用于训练，为None表示用所有交集基因进行映射

In [3]:
tg.pp_adatas(ad_sc,ad_sp,genes=None)

INFO:root:249 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:249 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [4]:
ad_map = tg.map_cells_to_space(ad_sc,ad_sp,device='cuda')

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 249 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Gene-voxel score: 0.103, Cell densities reg: 0.747


KeyboardInterrupt: 

返回的 AnnData（ad_map）是一个细胞-spot（voxel）结构，其中 ad_map.X[i, j] 表示细胞 i 位于spot j 的概率。<br>
这个ad_map可以用于将单细胞数据中的基因表达投射到空间中，这可以通过 tg.project_genes 实现。

In [ ]:
ad_sp.obs.head()

,orig.ident,nCount_RNA,nFeature_RNA,x,y,uniform_density,rna_count_based_density
ACCGGAACTTCTTC,0,14520.0,5453,3259.4,1233.7,0.000102,0.001352
GGGGGGGGGGGGGG,0,22554.0,8660,1662.6,2097.6,0.000102,0.003487
AAAGACCCAAAGGC,0,10774.0,4411,3050.3,1637.6,0.000102,0.001757
TCGGCGGTATCATT,0,10581.0,4033,1389.3,2045.8,0.000102,0.001987
TACGCCGGTACTGG,0,11075.0,4266,3022.8,1768.0,0.000102,0.001000


In [ ]:
ad_sp1 = sc.read_h5ad('/home/pxy/home/pxy/Tangram/data/test_ad_sp.h5ad')
ad_sp1.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'x', 'y', 'uniform_density',
       'rna_count_based_density'],
      dtype='object')

In [ ]:
# # 检查两个数据集的索引
# print("ad_map obs index:", ad_map.obs.index)
# print("ad_sc obs index:", ad_sc.obs.index)

# # 检查两个数据集的形状
print("ad_map shape:", ad_map.shape)
print("ad_sc shape:", ad_sc.shape)
print("ad_sp shape:", ad_sp.shape)

ad_map shape: (26431, 9852)
ad_sc shape: (26431, 249)
ad_sp shape: (9852, 249)


In [ ]:
# # 查看前5个细胞的名称
# print(ad_map.obs.index[:5])

# # 查看前5个空间点（spot）的名称
# print(ad_map.var.index[:5])

# # 查看第0个细胞在所有spot上的概率，并显示对应spot名
# cell_idx = 0
# cell_id = ad_map.obs.index[cell_idx]
# cell_probs = ad_map.X[cell_idx, :]
# for spot_idx, prob in enumerate(cell_probs):
#     spot_id = ad_map.var.index[spot_idx]
#     print(f"细胞 {cell_id} 在空间点 {spot_id} 的概率: {prob}")

Index(['pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4',
       'pBICCNsMMrMOpRPiM004d190318_TCAAGTGAGCACACAG-4',
       'pBICCNsMMrMOpRPiF008d190314_TTACGCCAGTCCCGAC-0',
       'pBICCNsMMrMOpRPiF005d190318_CTGCAGGCAGGCTATT-1',
       'pBICCNsMMrMOpRPiF006d190314_AATGCCACACTCTGCT-9'],
      dtype='object', name='index')
Index(['ACCGGAACTTCTTC', 'GGGGGGGGGGGGGG', 'AAAGACCCAAAGGC', 'TCGGCGGTATCATT',
       'TACGCCGGTACTGG'],
      dtype='object')
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 ACCGGAACTTCTTC 的概率: 1.1530685384286699e-07
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 GGGGGGGGGGGGGG 的概率: 2.689846405701246e-05
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 AAAGACCCAAAGGC 的概率: 2.886902450427442e-08
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 TCGGCGGTATCATT 的概率: 6.698616061839857e-08
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 TACGCCGGTACTGG 的概率: 9.687621371767818e-08
细胞 pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4 在空间点 TAAGTAGCAT

In [ ]:
ad_ge = tg.project_genes(adata_map=ad_map, adata_sc=ad_sc)

In [ ]:
# print(ad_sc.X.shape)
# print(ad_sc.obs.head())
# print(ad_sc.var.head())
# print(ad_sc.uns.keys())
# print(ad_sc.obsm.keys())

(26431, 249)
                                                    QC batch class_color  \
index                                                                      
pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4  PassQC     4     #00ADEE   
pBICCNsMMrMOpRPiM004d190318_TCAAGTGAGCACACAG-4  PassQC     4     #00ADEE   
pBICCNsMMrMOpRPiF008d190314_TTACGCCAGTCCCGAC-0  PassQC     0     #00ADEE   
pBICCNsMMrMOpRPiF005d190318_CTGCAGGCAGGCTATT-1  PassQC     1     #808080   
pBICCNsMMrMOpRPiF006d190314_AATGCCACACTCTGCT-9  PassQC     9     #808080   

                                                class_id    class_label  \
index                                                                     
pBICCNsMMrMOpRPiM004d190318_CATTGCCGTGAGAGGG-4         2  Glutamatergic   
pBICCNsMMrMOpRPiM004d190318_TCAAGTGAGCACACAG-4         2  Glutamatergic   
pBICCNsMMrMOpRPiF008d190314_TTACGCCAGTCCCGAC-0         2  Glutamatergic   
pBICCNsMMrMOpRPiF005d190318_CTGCAGGCAGGCTATT-1         3   Non-Neuronal   
pBIC